In [4]:
# payslip_universal_redactor.py — White masks, KEEP name>=60% (never redact),
# keep emp_id & dates, redact only values after {Address, Amount in words, Net Pay In {INR|USD|CAD}, DOB},
# plus free-form address block detection (bottom-left & below-name), safe merge that never crosses name,
# clickable Excel links

import argparse, sys, subprocess, pkgutil, re, json, traceback, shutil, difflib
from pathlib import Path
from typing import List, Tuple
import pandas as pd

# ----- deps -----
NEEDED = ("pymupdf", "pandas", "openpyxl", "pillow", "pytesseract")
def _pip_install(pkg: str):
    try:
        if pkg not in {m.name for m in pkgutil.iter_modules()}:
            print(f"[deps] Installing {pkg} ...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg], stdout=subprocess.DEVNULL)
    except Exception as e:
        print(f"[deps] Failed to install {pkg}: {e}")
        raise
for _p in NEEDED:
    _pip_install(_p)

import fitz  # PyMuPDF
from PIL import Image
import pytesseract
from openpyxl import load_workbook
from openpyxl.styles import Font

TESS_AVAILABLE = shutil.which("tesseract") is not None

# ---------- helpers ----------
def norm_word(s: str) -> str:
    """Lowercase, strip non-alnum."""
    return re.sub(r'[^a-z0-9]+', '', (s or '').lower())

def normalize_token(tok: str) -> str:
    return (tok or '').strip().lower().replace('\u00A0', ' ')

# ---------- date detection ----------
MONTHS = [
    "january","jan","february","feb","march","mar","april","apr","may","june","jun",
    "july","jul","august","aug","september","sep","sept","october","oct","november","nov","december","dec"
]
RE_DATE_ISO   = re.compile(r'^\d{4}[-/]\d{1,2}[-/]\d{1,2}$')               # 2024-09-25
RE_DATE_DMY   = re.compile(r'^\d{1,2}[-/]\d{1,2}[-/]\d{2,4}$')             # 25/09/2024
RE_DATE_DOTS1 = re.compile(r'^\d{4}\.\d{1,2}\.\d{1,2}$')                   # 2024.09.25
RE_DATE_DOTS2 = re.compile(r'^\d{1,2}\.\d{1,2}\.\d{2,4}$')                 # 25.09.2024
RE_MONTH_YEAR = re.compile(r'^(?:' + '|'.join(MONTHS) + r')\W*\d{2,4}$', re.I)
RE_YEAR_MONTH = re.compile(r'^\d{4}\W*(?:' + '|'.join(MONTHS) + r')$', re.I)
RE_RANGE_ONE  = re.compile(r'^\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\s*[-–]\s*\d{1,2}[-/]\d{1,2}[-/]\d{2,4}$', re.I)
RE_TIME       = re.compile(r'^\d{1,2}:\d{2}(:\d{2})?$')
RE_NUMERIC    = re.compile(r'\d')
RE_COMPACT_YMD = re.compile(r'^\d{8}$')  # 20250630
RE_YYYY = re.compile(r'^\d{4}$'); RE_MM = re.compile(r'^\d{2}$'); RE_DD = re.compile(r'^\d{2}$')

PAY_LABELS = {"payment date", "payment date:", "pay end date", "pay end date:"}

def looks_like_month_name(tok: str) -> bool:
    return normalize_token(tok).strip('.,') in MONTHS

def token_is_date(tok: str) -> bool:
    t = (tok or '').strip().strip(",.")
    if not t: return False
    if RE_DATE_ISO.match(t): return True
    if RE_DATE_DMY.match(t): return True
    if RE_DATE_DOTS1.match(t) or RE_DATE_DOTS2.match(t): return True
    if RE_RANGE_ONE.match(t): return True
    if RE_TIME.match(t): return True
    if looks_like_month_name(t): return True
    if RE_MONTH_YEAR.match(t) or RE_YEAR_MONTH.match(t): return True
    if RE_COMPACT_YMD.match(t): return True  # 20250630
    if re.search(r'^(?:' + '|'.join(MONTHS) + r')\W*\d{1,2}$', normalize_token(t), re.I): return True
    if re.search(r'^\d{1,2}\W*(?:' + '|'.join(MONTHS) + r')$', normalize_token(t), re.I): return True
    return False

def sequence_is_date(tokens: List[str], start_idx: int, max_len: int = 3) -> Tuple[bool,int]:
    # numeric triple like 2025 06 30
    if start_idx+2 < len(tokens):
        t0 = tokens[start_idx].strip().strip(",."); t1 = tokens[start_idx+1].strip().strip(",."); t2 = tokens[start_idx+2].strip().strip(",.")
        if RE_YYYY.match(t0) and RE_MM.match(t1) and RE_DD.match(t2):
            return True, 3
    for L in range(1, max_len+1):
        seq = tokens[start_idx:start_idx+L]
        if not seq: continue
        joined = " ".join(s.strip().strip(",.") for s in seq)
        jnorm = normalize_token(joined)
        if re.match(r'^\d{1,2}\s+(?:' + '|'.join(MONTHS) + r')(\s+\d{2,4})?$', jnorm, re.I): return True, L
        if re.match(r'^(?:' + '|'.join(MONTHS) + r')\s+\d{1,2}(\s*,?\s*\d{2,4})?$', jnorm, re.I): return True, L
        if RE_DATE_ISO.match(joined) or RE_DATE_DMY.match(joined) or RE_RANGE_ONE.match(joined): return True, L
    return False, 0

def in_pay_label_context(tokens: List[str], idx: int) -> bool:
    """Keep dates when near 'PAYMENT DATE' / 'PAY END DATE' labels."""
    start = max(0, idx-6)
    window = " ".join(normalize_token(t) for t in tokens[start:idx+1])
    for lbl in PAY_LABELS:
        if lbl in window:
            return True
    return False

# ---------- name / id ----------
def canonicalize_name_for_fuzzy(name: str) -> str:
    if not isinstance(name, str): return ""
    toks = re.findall(r"[A-Za-z0-9'-]+", name)
    return " ".join(t.lower() for t in toks if t.strip())

def name_match_ratio(tokens: List[str], idx: int, name_norm: str, min_ratio: float = 0.60) -> Tuple[bool,int]:
    """Fuzzy match window vs normalized Excel name, ignore single-letter initials."""
    if not name_norm: return (False, 0)
    name_tokens = name_norm.split()
    best_len, best_ratio = 0, 0.0
    max_len = min(len(tokens)-idx, len(name_tokens)+3)
    for L in range(1, max_len+1):
        window = tokens[idx:idx+L]
        cand_tokens = [norm_word(t) for t in window]
        cand_tokens = [c for c in cand_tokens if c and len(c) > 1]  # drop single-letter initials
        if not cand_tokens: continue
        cand_str = " ".join(cand_tokens)
        r = difflib.SequenceMatcher(None, cand_str, name_norm).ratio()
        if r > best_ratio:
            best_ratio, best_len = r, L
    return (best_ratio >= min_ratio and best_len > 0, best_len)

def empid_in_token(token: str, empid: str) -> bool:
    if not empid: return False
    tid = re.sub(r'\s+','', token or '')
    if empid in tid: return True
    simplified_token = re.sub(r'\W+','', token or '').lower()
    simplified_empid = re.sub(r'\W+','', empid).lower()
    return simplified_empid != "" and simplified_empid in simplified_token

# NEW: helper used by address-under-name logic
def fuzzy_contains_name(tokens: List[str], name_norm: str, min_ratio: float = 0.60) -> bool:
    """
    Returns True if the given token list (e.g., one line) fuzzy-matches the normalized
    employee name at >= min_ratio, ignoring single-letter initials.
    """
    if not name_norm or not tokens:
        return False
    cand_tokens = [norm_word(t) for t in tokens]
    cand_tokens = [c for c in cand_tokens if c and len(c) > 1]  # drop single-letter initials
    if not cand_tokens:
        return False
    cand_str = " ".join(cand_tokens)
    r = difflib.SequenceMatcher(None, cand_str, name_norm).ratio()
    return r >= min_ratio

# ---------- OCR/text words ----------
def get_words_text_or_ocr(page) -> List[tuple]:
    words = page.get_text("words")
    if words: return words
    if not TESS_AVAILABLE: return []
    # OCR fallback
    zoom = 300 / 72.0
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    ocr = pytesseract.image_to_data(img, output_type=pytesseract.Output.DATAFRAME)
    if ocr is None or ocr.empty: return []
    words_like = []
    page_rect = page.rect
    sx = page_rect.width / pix.width
    sy = page_rect.height / pix.height
    for _, row in ocr.iterrows():
        if str(row.get("text","")).strip()=="" or (isinstance(row.get("conf",0),(int,float)) and row["conf"]<0):
            continue
        left, top = row.get("left",0), row.get("top",0)
        width, height = row.get("width",0), row.get("height",0)
        text = str(row.get("text","")).strip()
        if not text: continue
        x0 = float(left)*sx + page_rect.x0
        y0 = float(top)*sy + page_rect.y0
        x1 = x0 + float(width)*sx
        y1 = y0 + float(height)*sy
        words_like.append((x0,y0,x1,y1,text,0,0,0))
    return words_like

# ---------- rect overlap & merge (safe around name) ----------
def rects_overlap(a: fitz.Rect, b: fitz.Rect) -> bool:
    return (min(a.x1, b.x1) - max(a.x0, b.x0) > 0) and (min(a.y1, b.y1) - max(a.y0, b.y0) > 0)

def can_merge_rects(a: fitz.Rect, b: fitz.Rect, gap_tol: float, protected: List[fitz.Rect]) -> bool:
    # same (approx) line, small horizontal gap
    v_overlap = min(a.y1, b.y1) - max(a.y0, b.y0)
    if v_overlap <= -gap_tol or b.x0 > a.x1 + gap_tol:
        return False
    # don't merge if the union would cross any protected (name) box
    union = fitz.Rect(min(a.x0, b.x0), min(a.y0, b.y0), max(a.x1, b.x1), max(a.y1, b.y1))
    for pb in protected:
        if rects_overlap(union, pb):
            if pb.x0 >= min(a.x0, b.x0) and pb.x1 <= max(a.x1, b.x1):
                return False
    return True

def merge_rects_safe(rects: List[fitz.Rect], protected: List[fitz.Rect], gap_tol: float = 2.0) -> List[fitz.Rect]:
    if not rects: return []
    rects_sorted = sorted(rects, key=lambda r: (round(r.y0,2), round(r.x0,2)))
    out = []
    cur = rects_sorted[0]
    for r in rects_sorted[1:]:
        if can_merge_rects(cur, r, gap_tol, protected):
            cur = fitz.Rect(min(cur.x0, r.x0), min(cur.y0, r.y0), max(cur.x1, r.x1), max(cur.y1, r.y1))
        else:
            out.append(cur); cur = r
    out.append(cur)
    return out

# ---------- label -> value redaction (keep label visible) ----------
def split_label_tokens(label: str) -> List[str]:
    return [norm_word(t) for t in re.findall(r"[A-Za-z0-9]+", label)]

def match_label_at(tokens_norm: List[str], i: int, label_tokens: List[str]) -> int:
    L = len(label_tokens)
    if L == 0 or i+L > len(tokens_norm): return 0
    return L if tokens_norm[i:i+L] == label_tokens else 0

ADDRESS_LABELS = ["address", "address:", "residential address", "home address", "mailing address"]
AMOUNT_WORDS_LABELS = ["amount in words", "amount in words:"]
NETPAY_LABELS = ["net pay in inr", "net pay in usd", "net pay in cad"]
DOB_LABELS = ["dob", "d.o.b", "date of birth", "birth date", "birthdate"]
LABEL_GROUPS = ADDRESS_LABELS + AMOUNT_WORDS_LABELS + NETPAY_LABELS + DOB_LABELS
LABEL_TOKENS = [(lab, split_label_tokens(lab)) for lab in LABEL_GROUPS]

def collect_value_after_label_rects(words_sorted: List[tuple], label_tokens: List[Tuple[str, List[str]]]) -> List[fitz.Rect]:
    """Redact ONLY the value to the right of the label on the same line (label stays visible)."""
    rects = []
    tokens = [w[4] for w in words_sorted]
    tokens_norm = [norm_word(t) for t in tokens]
    i = 0
    N = len(tokens)
    while i < N:
        matched = False
        for lab, lab_tok in label_tokens:
            L = match_label_at(tokens_norm, i, lab_tok)
            if L > 0:
                y0 = words_sorted[i][1]
                last_x1 = words_sorted[i+L-1][2]
                k = i + L
                while k < N and abs(words_sorted[k][1] - y0) < 2.5:
                    if words_sorted[k][0] >= last_x1 - 0.5:
                        w = words_sorted[k]
                        rects.append(fitz.Rect(w[0], w[1], w[2], w[3]))
                    k += 1
                i += L
                matched = True
                break
        if not matched:
            i += 1
    return rects

# ---------- free-form address block detection (bottom-left & below-name) ----------
STREET_KEYWORDS = {
    "st", "street", "rd", "road", "ave", "avenue", "blvd", "drive", "dr", "lane", "ln",
    "court", "ct", "terrace", "way", "cres", "crescent", "parkway", "pkwy", "highway", "hwy"
}
UNIT_KEYWORDS = {"unit", "apt", "apartment", "suite", "ste", "floor", "fl", "flat"}
RE_STREET_LINE = re.compile(r'^\s*\d+[A-Za-z]?\s+[A-Za-z0-9\'\.\- ]{2,}$')
RE_CAN_POSTAL  = re.compile(r'[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d')
RE_US_ZIP      = re.compile(r'\b\d{5}(-\d{4})?\b')

def group_lines(words_sorted: List[tuple], y_tol: float = 2.5):
    """Group tokens into line objects with text and bounding rect."""
    lines = []
    if not words_sorted:
        return lines
    cur = [words_sorted[0]]
    for w in words_sorted[1:]:
        if abs(w[1] - cur[-1][1]) < y_tol:
            cur.append(w)
        else:
            lines.append(cur)
            cur = [w]
    lines.append(cur)
    out = []
    for line in lines:
        xs0 = [w[0] for w in line]; ys0 = [w[1] for w in line]; xs1 = [w[2] for w in line]; ys1 = [w[3] for w in line]
        text = " ".join(w[4] for w in line)
        rect = fitz.Rect(min(xs0), min(ys0), max(xs1), max(ys1))
        out.append({"rect": rect, "text": text, "words": line})
    return out

# Add this new regex near your other globals (with the other RE_* constants):
RE_SHORT_UPPER = re.compile(r"^[A-Z][A-Z\s/.\-'/]{0,24}$")  # short, all-caps fragments like "SHERBOURNE", "E ST", "N"

def is_address_like(text: str) -> bool:
    """
    Strong signal a single line is address-like. Kept intentionally *broad* for bottom-left stubs.
    """
    t = normalize_token(text)
    # Already-strong signals you had
    if RE_STREET_LINE.search(text): return True
    if RE_CAN_POSTAL.search(text) or RE_US_ZIP.search(text): return True

    # Presence of common street/unit tokens anywhere on the line
    toks = [norm_word(x) for x in re.findall(r"[A-Za-z0-9']+", text)]
    if any(tok in STREET_KEYWORDS for tok in toks): return True
    if any(tok in UNIT_KEYWORDS for tok in toks): return True

    # NEW: short, all-caps fragments common in tear-off stubs (e.g., "SHERBOURNE", "E ST", single-letter directions)
    # We treat this as a *weak* address signal; the cluster logic below will require ≥2 such weak lines or one strong neighbor.
    if RE_SHORT_UPPER.match(text) and len(toks) <= 3:
        return True

    return False

def collect_freeform_address_rects(page, words_sorted: List[tuple], name_norm: str) -> List[fitz.Rect]:
    """
    Redact free-form address blocks that appear:
      - in the few lines under the detected name (already in your logic), and
      - in the bottom-left stub area where addresses are often broken into short all-caps lines.
    Uses a small cluster heuristic so that weak lines like "E ST" or "N" are captured
    when adjacent to another weak/strong address-like line. Never touches name lines.
    """
    rects = []
    page_rect = page.rect

    # --- group into lines first (reuses your helper) ---
    lines = group_lines(words_sorted)

    # --- strengthened "under name" behavior (captures SHERBOURNE / N stubs) ---
    name_line_idxs = []
    for idx, L in enumerate(lines):
        tokens = [w[4] for w in L["words"]]
        if fuzzy_contains_name(tokens, name_norm, 0.60):
            name_line_idxs.append(idx)

    for idx in name_line_idxs:
        # Look at up to the next 6 lines and build a consecutive block of address-like lines.
        candidate_rects = []
        for j in range(idx + 1, min(idx + 7, len(lines))):
            line = lines[j]
            line_tokens = [w[4] for w in line["words"]]

            # Never touch name lines
            if fuzzy_contains_name(line_tokens, name_norm, 0.60):
                break

            txt = line["text"]
            # Treat as address-like if:
            #  - our existing strong detector says so, OR
            #  - it's a short all-caps fragment (RE_SHORT_UPPER), OR
            #  - it contains any digit (common with unit/house numbers)
            is_addrish = (
                is_address_like(txt)
                or RE_SHORT_UPPER.match(txt) is not None
                or bool(re.search(r"\d", txt))
            )

            # We collect *consecutive* address-like lines; stop on the first non-address line.
            if is_addrish:
                candidate_rects.append(line["rect"])
            else:
                break

        # If we collected anything, redact as a single tidy block
        if candidate_rects:
            r = fitz.Rect(
                min(rr.x0 for rr in candidate_rects),
                min(rr.y0 for rr in candidate_rects),
                max(rr.x1 for rr in candidate_rects),
                max(rr.y1 for rr in candidate_rects),
            )
            rects.append(r)

    # --- bottom-left candidate lines ---
    bottom_left = []
    for line in lines:
        # bottom ~58% of page, left ~65% width — slightly wider than before to catch far-left text
        midy = 0.5 * (line["rect"].y0 + line["rect"].y1)
        if midy >= page_rect.y0 + 0.42 * page_rect.height and line["rect"].x0 <= page_rect.x0 + 0.65 * page_rect.width:
            bottom_left.append(line)

    if not bottom_left:
        return rects

    # --- cluster adjacent lines by small vertical gaps (so "SHERBOURNE" + "N" get grouped) ---
    def cluster_lines(lines_in, max_gap=7.0):
        clusters = []
        cur = [lines_in[0]]
        for L in lines_in[1:]:
            if abs(L["rect"].y0 - cur[-1]["rect"].y0) < max_gap:
                cur.append(L)
            else:
                clusters.append(cur); cur = [L]
        clusters.append(cur)
        return clusters

    clusters = cluster_lines(bottom_left, max_gap=7.0)

    # --- decide cluster-level addressness ---
    for cluster in clusters:
        # Count lines with strong signals vs weak all-caps snippets
        strong = 0
        weak_caps = 0
        has_digit = 0
        for L in cluster:
            txt = L["text"]
            if is_address_like(txt):
                strong += 1
            if RE_SHORT_UPPER.match(txt):
                weak_caps += 1
            if re.search(r"\d", txt):
                has_digit += 1

        # Cluster is address if:
        #   (a) any strong line present, or
        #   (b) at least two weak all-caps fragments, or
        #   (c) one weak all-caps fragment + a neighbor that contains digits (house/unit number)
        cluster_is_address = (strong >= 1) or (weak_caps >= 2) or (weak_caps >= 1 and has_digit >= 1)

        if not cluster_is_address:
            continue

        # Build a union rect of all *non-name* lines in the cluster
        cluster_rects = []
        for L in cluster:
            if not fuzzy_contains_name([w[4] for w in L["words"]], name_norm, 0.60):
                cluster_rects.append(L["rect"])

        if not cluster_rects:
            continue

        # Join into a single rectangular block (cleaner coverage)
        r = fitz.Rect(
            min(rr.x0 for rr in cluster_rects),
            min(rr.y0 for rr in cluster_rects),
            max(rr.x1 for rr in cluster_rects),
            max(rr.y1 for rr in cluster_rects),
        )
        rects.append(r)

    return rects


# ---------- main targeting ----------
def tokens_to_redact(page, name_norm: str, empid: str) -> List[fitz.Rect]:
    words = get_words_text_or_ocr(page)
    if not words: return []
    words_sorted = sorted(words, key=lambda w: (round(w[1],1), round(w[0],1)))
    tokens = [w[4] for w in words_sorted]
    tokens_norm = [norm_word(t) for t in tokens]

    rects: List[fitz.Rect] = []
    name_boxes: List[fitz.Rect] = []  # protect these from overlap/merge

    # A) Protect all name occurrences (we never redact names)
    i = 0; N = len(tokens)
    while i < N:
        keep_name, name_len = name_match_ratio(tokens, i, name_norm, 0.60)
        if keep_name and name_len > 0:
            for kk in range(i, i+name_len):
                w = words_sorted[kk]
                name_boxes.append(fitz.Rect(w[0], w[1], w[2], w[3]))
            i += name_len
        else:
            i += 1

    # B) Label → value redactions (label text itself stays)
    rects += collect_value_after_label_rects(words_sorted, LABEL_TOKENS)

    # C) Free-form address blocks (bottom-left & lines below name)
    rects += collect_freeform_address_rects(page, words_sorted, name_norm)

    # D) Numeric redactions (keep dates and emp_id)
    i = 0
    while i < N:
        # keep multi-token dates
        is_seq_date, seq_len = sequence_is_date(tokens, i, max_len=3)
        if is_seq_date:
            i += seq_len
            continue
        tok = tokens[i].strip()
        if RE_NUMERIC.search(tok):
            # keep single-token dates or dates near pay labels
            if token_is_date(tok) or in_pay_label_context(tokens, i):
                i += 1
                continue
            # keep emp_id
            if empid and empid_in_token(tok, empid):
                i += 1
                continue
            # redact numeric token
            w = words_sorted[i]
            rects.append(fitz.Rect(w[0], w[1], w[2], w[3]))
        i += 1

    # E) Remove any box that overlaps the protected name boxes, then merge safely
    rects = [r for r in rects if not any(rects_overlap(r, nb) for nb in name_boxes)]
    rects = merge_rects_safe(rects, protected=name_boxes, gap_tol=2.0)
    return rects

def redact_page(page, rects: List[fitz.Rect], fill=(1,1,1)):
    """WHITE masks as requested."""
    if not rects: return 0
    for r in rects:
        pad = 0.8
        rr = fitz.Rect(r.x0 - pad, r.y0 - pad, r.x1 + pad, r.y1 + pad)
        page.add_redact_annot(rr, fill=fill)
    page.apply_redactions()
    return len(rects)

def process_pdf(pdf_path: Path, emp_name: str, empid: str, out_path: Path, verbose: bool=False) -> Tuple[bool,str]:
    try:
        doc = fitz.open(str(pdf_path))
    except Exception as e:
        return False, f"ERROR opening PDF: {e}"

    name_norm = canonicalize_name_for_fuzzy(emp_name)
    any_redactions = 0
    for pno in range(len(doc)):
        page = doc[pno]
        rects = tokens_to_redact(page, name_norm=name_norm, empid=str(empid or ""))
        n = redact_page(page, rects, fill=(1,1,1))
        any_redactions += n
        if verbose:
            print(f"  page {pno+1}: redactions={n}")

    try:
        out_path.parent.mkdir(parents=True, exist_ok=True)
        doc.save(str(out_path))
        doc.close()
        return True, f"OK, redactions={any_redactions}"
    except Exception as e:
        return False, f"ERROR saving redacted PDF: {e}"

# ---------- Excel helpers ----------
def detect_columns(df: pd.DataFrame):
    """Robust detection of emp_id, emp_name, payslip link columns."""
    norm_to_orig = {str(c).strip().lower(): c for c in df.columns}
    empid_col = None; empname_col = None; payslip_col = None
    empid_keys   = {"empid","emp_id","employeeid","employee_id","id"}
    empname_keys = {"empname","emp_name","employee","employee_name","name"}
    payslip_keys = {"payslip","payslip_link","paysliplink","payslip_path","file","file_path","pdf","payslip-link"}
    for k, orig in norm_to_orig.items():
        if empid_col   is None and k in empid_keys:   empid_col   = orig
        if empname_col is None and k in empname_keys: empname_col = orig
        if payslip_col is None and k in payslip_keys: payslip_col = orig
    if payslip_col is None:
        for orig in df.columns:
            ser = df[orig].astype(str)
            if ser.str.contains(r'\.pdf($|\?)', case=False, na=False).any() or \
               ser.str.contains(r'^https?://', case=False, na=False).any():
                payslip_col = orig; break
    return empid_col, empname_col, payslip_col

def hyperlinkize_output(excel_path: Path, link_column_name: str = "payslip_redacted_link"):
    """Make the link column clickable (file:///...)."""
    try:
        wb = load_workbook(excel_path)
        ws = wb.active
        header = [c.value for c in next(ws.iter_rows(min_row=1, max_row=1))]
        if link_column_name not in header:
            wb.save(excel_path); return
        col_idx = header.index(link_column_name) + 1
        for row in ws.iter_rows(min_row=2):
            cell = row[col_idx-1]
            val = cell.value
            if not isinstance(val, str): continue
            if val.startswith("ERROR"): continue
            p = Path(val)
            uri = ("file:///" + str(p.resolve()).replace("\\","/"))
            cell.hyperlink = uri
            cell.value = str(p.resolve())
            cell.font = Font(color="0563C1", underline="single")
        wb.save(excel_path)
    except Exception as e:
        print(f"[WARN] Could not convert links to hyperlinks: {e}")

def write_helper_notebook(folder: Path, input_excel: Path):
    nb = {
      "cells": [{
          "cell_type": "code",
          "metadata": {},
          "source": [
            "# Run the universal payslip redactor\n",
            "!python -m pip install pymupdf pandas openpyxl pillow pytesseract\n",
            f"!python payslip_universal_redactor.py --input \"{input_excel.name}\" --verbose\n"
          ],
          "outputs": [], "execution_count": None
      }],
      "metadata": {"kernelspec": {"display_name":"Python 3","language":"python","name":"python3"},
                   "language_info": {"name":"python"}},
      "nbformat": 4, "nbformat_minor": 5
    }
    path = folder / "payslip_universal_redactor.ipynb"
    try:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(nb, f, ensure_ascii=False, indent=2)
        return True, str(path)
    except Exception as e:
        return False, str(e)

# ---------- main ----------
def main(argv=None):
    p = argparse.ArgumentParser(description="Payslip Redactor (white masks, keep name>=60%, keep emp_id & dates, label-value + address-block redaction, safe merge)")
    p.add_argument("--input","-i", default="Payslips.xlsx", help="Input Excel/CSV. Default: Payslips.xlsx")
    p.add_argument("--sheet","-s", default=None, help="Sheet name (Excel)")
    p.add_argument("--out","-o", default=None, help="Output Excel filename (default beside input)")
    p.add_argument("--no-notebook", action="store_true", help="Do not write helper .ipynb")
    p.add_argument("--verbose","-v", action="store_true", help="Verbose logging")
    # Accept notebook-injected flags
    args, unknown = p.parse_known_args(argv if argv is not None else sys.argv[1:])
    if unknown:
        print(f"[WARN] Ignoring unknown args: {unknown}")

    in_path = Path(args.input).expanduser().resolve()
    if not in_path.exists():
        print(f"[ERROR] Input file not found: {in_path}")
        return 2

    # read input (ALWAYS DataFrame)
    try:
        if in_path.suffix.lower() in (".xls", ".xlsx"):
            if args.sheet is None:
                df = pd.read_excel(in_path, sheet_name=0, engine="openpyxl")
            else:
                df = pd.read_excel(in_path, sheet_name=args.sheet, engine="openpyxl")
            if isinstance(df, dict):
                if args.sheet and args.sheet in df:
                    df = df[args.sheet]
                else:
                    df = next(iter(df.values()))
        elif in_path.suffix.lower() == ".csv":
            df = pd.read_csv(in_path)
        else:
            print("[ERROR] Unsupported input type. Use .xlsx or .csv")
            return 2
    except Exception as e:
        print(f"[ERROR] Reading input failed: {e}")
        return 2

    empid_col, empname_col, payslip_col = detect_columns(df)
    if not payslip_col:
        print("[ERROR] Could not detect the payslip path/URL column.")
        print("Columns:", list(df.columns))
        return 2
    if not empname_col:
        print("[WARNING] Employee name column not detected — name keep may miss.")
    if not empid_col:
        print("[WARNING] Employee id column not detected — emp_id may be redacted.")

    out_excel = Path(args.out).expanduser().resolve() if args.out else (in_path.parent / "Payslips_redacted.xlsx")

    if not args.no_notebook:
        ok, msg = write_helper_notebook(in_path.parent, in_path)
        print(f"[INFO] Wrote helper notebook: {msg}" if ok else f"[WARN] Could not write notebook: {msg}")

    results = []
    total = len(df)
    print(f"[INFO] Tesseract OCR available: {'YES' if TESS_AVAILABLE else 'NO'}")
    print(f"[INFO] Processing {total} rows...")

    for ridx, row in df.iterrows():
        empid = str(row[empid_col]) if empid_col in row.index and not pd.isna(row[empid_col]) else ""
        emp_name = str(row[empname_col]) if empname_col in row.index and not pd.isna(row[empname_col]) else ""
        pdf_ref = row[payslip_col]

        if pd.isna(pdf_ref):
            results.append({**row.to_dict(), "payslip_redacted_link": "ERROR: missing payslip link"})
            print(f"[{ridx+1}/{total}] Missing payslip link; skipping")
            continue

        pdf_path = Path(str(pdf_ref)).expanduser()
        if not pdf_path.is_absolute():
            pdf_path = (in_path.parent / pdf_path).resolve()
        if not pdf_path.exists():
            results.append({**row.to_dict(), "payslip_redacted_link": f"ERROR: pdf not found ({pdf_path})"})
            print(f"[{ridx+1}/{total}] PDF not found: {pdf_path}")
            continue

        out_path = pdf_path.with_name(pdf_path.stem + "_redacted" + pdf_path.suffix)

        try:
            success, msg = process_pdf(pdf_path, emp_name, empid, out_path, verbose=args.verbose)
            if success:
                results.append({**row.to_dict(), "payslip_redacted_link": str(out_path)})
                print(f"[{ridx+1}/{total}] REDACTED: {out_path.name} ({msg})")
            else:
                results.append({**row.to_dict(), "payslip_redacted_link": f"ERROR: {msg}"})
                print(f"[{ridx+1}/{total}] FAILED: {msg}")
        except Exception as e:
            tb = traceback.format_exc()
            results.append({**row.to_dict(), "payslip_redacted_link": f"ERROR: {e}"})
            print(f"[{ridx+1}/{total}] Exception: {e}\n{tb}")

    out_df = pd.DataFrame(results)
    try:
        out_df.to_excel(out_excel, index=False, engine="openpyxl")
        hyperlinkize_output(out_excel, link_column_name="payslip_redacted_link")
        print(f"[INFO] Results written to {out_excel}")
    except Exception as e:
        print(f"[ERROR] Writing output Excel failed: {e}")
        return 3

    print("[DONE]")
    return 0

if __name__ == "__main__":
    try:
        get_ipython  # notebook present
        main(None)   # run without SystemExit in notebooks
    except NameError:
        raise SystemExit(main(None))  # normal scripts


[deps] Installing pillow ...
[WARN] Ignoring unknown args: ['-f', 'C:\\Users\\highm\\AppData\\Roaming\\jupyter\\runtime\\kernel-8f5e7972-3101-457d-97d2-f61d84e15548.json']
[INFO] Wrote helper notebook: C:\Users\highm\Downloads\Sample_Payslips\payslip_universal_redactor.ipynb
[INFO] Tesseract OCR available: NO
[INFO] Processing 4 rows...
[1/4] REDACTED: Mandeep_payslip_redacted.pdf (OK, redactions=38)
[2/4] REDACTED: Suraj_payslip_redacted.pdf (OK, redactions=51)
[3/4] REDACTED: Rajesh_payslip_redacted.pdf (OK, redactions=20)
[4/4] REDACTED: Harry_payslip_redacted.pdf (OK, redactions=18)
[ERROR] Writing output Excel failed: [Errno 13] Permission denied: 'C:\\Users\\highm\\Downloads\\Sample_Payslips\\Payslips_redacted.xlsx'
